In [80]:
from simtk.openmm.app import*
from openforcefield.utils import*

In [81]:
from simtk.openmm.app import PDBFile
from openforcefield.utils import get_data_file_path

In [82]:
ls

Acetylacetone_04_July.py       Acetylacetone.pdb  traj.dcd
Acetylacetone Exp-Copy1.ipynb  data.csv           trajectory.pdb
Acetylacetone Exp-Copy.ipynb   ff_valuefile.txt
Acetylacetone_mdtraj.ipynb     README.md


In [83]:
import os

pdb_file_path = os.path.join('Acetylacetone.pdb')
pdbfile = PDBFile(pdb_file_path)

# I got the pdb file from
# https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/P2D
# renamed it, then I manually moved it to the data directory of openforcefield.

print(pdbfile)

In [84]:
from openforcefield.topology import Molecule
acetylacetone = Molecule.from_smiles('O=C(C)CC(=O)C')
print(acetylacetone)

# I got the SMILES-code from:
# https://en.wikipedia.org/wiki/Acetylacetone

Molecule with name '' and SMILES '[H][C]([H])([H])[C](=[O])[C]([H])([H])[C](=[O])[C]([H])([H])[H]'


In [85]:
print (pdb_file_path)

Acetylacetone.pdb


In [86]:
from openforcefield.topology import *
from openforcefield.typing.engines.smirnoff import ForceField

omm_topology =  pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[acetylacetone])

forcefield = ForceField('openff-1.0.0.offxml')

system = forcefield.create_openmm_system(off_topology)


In [87]:
from simtk import *

# Langevin Dynamics:
time_step = 2*unit.femtoseconds
temperature = 300*unit.kelvin
friction = 1/unit.picosecond # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Lenghth of simulation:
num_steps = 1

trj_freq = 1 # number of steps per written trajectory frame
data_freq = 1 #number of steps per fritten simulation statistics

# Set up OpenMM simulation:
simulation = openmm.app.Simulation (omm_topology, system, integrator)

# initial positions:
positions = pdbfile.getPositions()
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# MINIMIZE ENERGY: I deleted this line to get the non-minimized values:
simulation.minimizeEnergy()
minimized_coords = simulation.context.getState(getPositions=True).getPositions()
minimized_forces = simulation.context.getState(getForces=True).getForces()

# Configure information in output files:
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter ('data.csv', data_freq, step=True, potentialEnergy=True, temperature=True, density=True)
simulation.reporters.append (pdb_reporter)
simulation.reporters.append (state_data_reporter)
DCDReporter = openmm.app.DCDReporter ('traj.dcd', reportInterval=1)
simulation.reporters.append (DCDReporter)

In [88]:
help (openmm.app.DCDReporter)

Help on class DCDReporter in module simtk.openmm.app.dcdreporter:

class DCDReporter(builtins.object)
 |  DCDReporter(file, reportInterval, append=False, enforcePeriodicBox=None)
 |  
 |  DCDReporter outputs a series of frames from a Simulation to a DCD file.
 |  
 |  To use it, create a DCDReporter, then add it to the Simulation's list of reporters.
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __init__(self, file, reportInterval, append=False, enforcePeriodicBox=None)
 |      Create a DCDReporter.
 |      
 |      Parameters
 |      ----------
 |      file : string
 |          The file to write to
 |      reportInterval : int
 |          The interval (in time steps) at which to write frames
 |      append : bool=False
 |          If True, open an existing DCD file to append to.  If False, create a new file.
 |      enforcePeriodicBox: bool
 |          Specifies whether particle positions should be translated so the center of every molecule
 |          lies in the sam

In [89]:
import time

print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Hurra!")

Starting simulation
Elapsed time 0.00 seconds
Hurra!


In [90]:
ff_applied_parameters = forcefield.label_molecules(off_topology)[0]
ff_values=[]
ff_valuefile = open ('ff_valuefile.txt', 'w+')

for atoms, bonds in ff_applied_parameters['Bonds'].items():
    ff_valuefile.write (F'{atoms},{bonds}')
    ff_valuefile.write ('\n')
    
ff_valuefile.close()

In [91]:
import numpy

ff_valuefile = open ('ff_valuefile.txt', 'r')

ff_valuetab = numpy.genfromtxt(fname=ff_valuefile, dtype='unicode', delimiter='Lenght')

ff_valuefile.close

man_atom_a = [0,1,1,2,2,2,3,3,3,4,4,6,6,6]
man_atom_b = [1,2,3,7,8,9,4,10,11,5,6,12,13,14]

In [92]:
import numpy
import os

mincoorfile = os.path.join('trajectory.pdb')

mincoor = numpy.genfromtxt(fname=mincoorfile, skip_header=2, skip_footer=2, dtype='unicode')

print (mincoor)

[['HETATM' '1' 'C1' 'P2D' 'A' '1' '2.269' '-0.174' '0.040' '1.00' '0.00'
  'C']
 ['HETATM' '2' 'C2' 'P2D' 'A' '1' '1.037' '-0.839' '-0.571' '1.00' '0.00'
  'C']
 ['HETATM' '3' 'C3' 'P2D' 'A' '1' '-0.012' '-1.475' '0.313' '1.00' '0.00'
  'C']
 ['HETATM' '4' 'C4' 'P2D' 'A' '1' '-1.010' '-0.386' '0.588' '1.00' '0.00'
  'C']
 ['HETATM' '5' 'C5' 'P2D' 'A' '1' '-2.245' '-0.210' '-0.272' '1.00'
  '0.00' 'C']
 ['HETATM' '6' 'O2' 'P2D' 'A' '1' '0.847' '-0.823' '-1.768' '1.00' '0.00'
  'O']
 ['HETATM' '7' 'O4' 'P2D' 'A' '1' '-0.812' '0.379' '1.534' '1.00' '0.00'
  'O']
 ['HETATM' '8' 'H1' 'P2D' 'A' '1' '2.762' '-0.913' '0.677' '1.00' '0.00'
  'H']
 ['HETATM' '9' 'H2' 'P2D' 'A' '1' '2.904' '0.181' '-0.776' '1.00' '0.00'
  'H']
 ['HETATM' '10' 'H3' 'P2D' 'A' '1' '1.958' '0.690' '0.632' '1.00' '0.00'
  'H']
 ['HETATM' '11' 'H4' 'P2D' 'A' '1' '-0.444' '-2.312' '-0.242' '1.00'
  '0.00' 'H']
 ['HETATM' '12' 'H5' 'P2D' 'A' '1' '0.434' '-1.768' '1.267' '1.00' '0.00'
  'H']
 ['HETATM' '13' 'H6' 'P2D' 'A'

In [93]:
xyz_data = mincoor[:,6:9]
xyz_data = xyz_data.astype(numpy.float)
print (xyz_data)

xa = xyz_data[man_atom_a,0]
xb = xyz_data[man_atom_b,0]
ya = xyz_data[man_atom_a,1]
yb = xyz_data[man_atom_b,1]
za = xyz_data[man_atom_a,2]
zb = xyz_data[man_atom_b,2]

[[ 2.269 -0.174  0.04 ]
 [ 1.037 -0.839 -0.571]
 [-0.012 -1.475  0.313]
 [-1.01  -0.386  0.588]
 [-2.245 -0.21  -0.272]
 [ 0.847 -0.823 -1.768]
 [-0.812  0.379  1.534]
 [ 2.762 -0.913  0.677]
 [ 2.904  0.181 -0.776]
 [ 1.958  0.69   0.632]
 [-0.444 -2.312 -0.242]
 [ 0.434 -1.768  1.267]
 [-2.832 -1.13  -0.202]
 [-2.831  0.625  0.118]
 [-1.929 -0.067 -1.308]]


In [94]:
bondlength_min=numpy.sqrt(((xa-xb)**2)+((ya-yb)**2)+((za-zb)**2))

print (bondlength_min)    

[1.52753723 1.51206911 2.39555818 2.85366711 3.52580672 2.94446701
 1.51519009 2.17226426 2.11094789 3.48916165 2.37950541 3.06123782
 2.47829236 3.08602803]


In [95]:
import mdtraj as md

t = md.load('traj.dcd', top='Acetylacetone.pdb')
print(t)

<mdtraj.Trajectory with 1 frames, 15 atoms, 1 residues, without unitcells>


In [96]:
from __future__ import print_function
print('How many atoms? %s' %t.n_atoms)

How many atoms? 15


In [154]:
frame_idx = 0
atom_idx = 9
print('Where is 9th Atom?')
print('x; %s\ty: %s\tz: %s' %tuple(t.xyz[frame_idx, atom_idx,:]))

Where is 9th Atom?
x; 0.19575684	y: 0.069006205	z: 0.063209236


In [181]:
t_topology=t.topology
print(t_topology)

for atom in t_topology.atoms:
    print (atom)

<mdtraj.Topology with 1 chains, 1 residues, 15 atoms, 14 bonds>


TypeError: descriptor 'count' requires a 'list' object but received a 'module'

In [200]:
import numpy as np

#atom_pairslist= np.array(
#[[0, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 6, 6, 6], 
 #[1, 2, 3, 7, 8, 9, 4, 10, 11, 5, 6, 12, 13, 14]])
#print(atom_pairslist)

COlist = numpy.array ([[1,5],[3,6]])
CClist = numpy.array ([[0,1], [1,2],[2,3], [3,4]])
C1Hlist = numpy.array ([[0,7], [0,8], [0,9]])
C3Hlist = numpy.array ([[2,10], [2,11]])
C5Hlist = numpy.array ([[4,12], [4,13], [4,14]])

In [212]:
CO_distances=(md.compute_distances(t, COlist))
print (CO_distances)

[[0.12119107 0.12332539]]


In [211]:
CC_distances = md.compute_distances(t, CClist)
print (CC_distances)

[[0.15278436 0.15121521 0.15030983 0.15145439]]


In [210]:
C1H_distances = md.compute_distances(t, C1Hlist)
print (C1H_distances)

[[0.10928881 0.10928891 0.10928883]]


In [209]:
C3H_distances=md.compute_distances(t, C3Hlist)
print (C3H_distances)

[[0.10928919 0.10928883]]


In [208]:
C5H_distances = md.compute_distances(t, C5Hlist)
print (C5H_distances)

[[0.10928886 0.10928883 0.10928891]]
